# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [3]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [4]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = turicreate.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 1.195780     | 4146407.600631     | 258679.804477                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [5]:
example_weight_summary = example_model.coefficients
print (example_weight_summary)

+-------------+-------+--------------------+--------------------+
|     name    | index |       value        |       stderr       |
+-------------+-------+--------------------+--------------------+
| (intercept) |  None |  87910.0724923751  | 7873.338143401263  |
| sqft_living |  None | 315.4034405520954  | 3.4557003258546763 |
|   bedrooms  |  None | -65080.21555282187 | 2717.4568544206145 |
|  bathrooms  |  None | 6944.020192643775  | 3923.114931441477  |
+-------------+-------+--------------------+--------------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [7]:
example_predictions = example_model.predict(train_data)
print (example_predictions[0]) # should be 271789.505878

271789.50587802584


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [40]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions

    # Then compute the residuals/errors

    # Then square and add them up
    
    y_pred = model.predict(data)
    RSS = ((outcome - y_pred)**2).sum()

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [35]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print ("{:.10e}".format(rss_example_train)) # should be 2.7376153833e+14

2.7376153833e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [26]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [27]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

Using default 16 lambda workers.

To maximize the degree of parallelism, add the following code to the beginning of the program:

"turicreate.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 24)"

Note that increasing the degree of parallelism also increases the memory footprint.

In [28]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [30]:
avg_bs = test_data['bedrooms_squared'].mean()
avg_bbr = test_data['bed_bath_rooms'].mean()
avg_lsl = test_data['log_sqft_living'].mean()
avg_lpl = test_data['lat_plus_long'].mean()
print("Average of bedrooms_squared: ", avg_bs)
print("\nAverage of bed_bath_rooms: ", avg_bbr)
print("\nAverage of log_sqft_living: ", avg_lsl)
print("\nAverage of lat_plus_long: ", avg_lpl)

Average of bedrooms_squared:  12.446677701584298

Average of bed_bath_rooms:  7.5039016315913925

Average of log_sqft_living:  7.550274679645938

Average of lat_plus_long:  -74.65333497217313


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [31]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [32]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = turicreate.linear_regression.create(train_data, target = 'price', \
                                              features = model_1_features, \
                                              validation_set = None)
model_2 = turicreate.linear_regression.create(train_data, target = 'price', \
                                              features = model_2_features, \
                                              validation_set = None)
model_3 = turicreate.linear_regression.create(train_data, target = 'price', \
                                              features = model_3_features, \
                                              validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.105509     | 3193229.177879     | 228200.043155                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [38]:
# Examine/extract each model's coefficients:
model_1_weight_summary = model_1.coefficients
print (model_1_weight_summary)

model_2_weight_summary = model_2.coefficients
print (model_2_weight_summary)

model_3_weight_summary = model_3.coefficients
print (model_3_weight_summary)

+-------------+-------+---------------------+--------------------+
|     name    | index |        value        |       stderr       |
+-------------+-------+---------------------+--------------------+
| (intercept) |  None |  -56140675.7506105  | 1649985.4205999067 |
| sqft_living |  None |  310.2633257790604  | 3.1888296040766235 |
|   bedrooms  |  None |  -59577.11606709479 | 2487.279773218164  |
|  bathrooms  |  None |  13811.840542854276 | 3593.542132963566  |
|     lat     |  None |  629865.7894558223  | 13120.710032291865 |
|     long    |  None | -214790.28524819054 | 13284.285163302153 |
+-------------+-------+---------------------+--------------------+
[6 rows x 4 columns]

+----------------+-------+---------------------+--------------------+
|      name      | index |        value        |       stderr       |
+----------------+-------+---------------------+--------------------+
|  (intercept)   |  None | -54410676.120700724 | 1650405.1657380708 |
|  sqft_living   |  None |  

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [41]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
model_1_rss = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
model_2_rss = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
model_3_rss = get_residual_sum_of_squares(model_3, train_data, train_data['price'])

print("RSS of Model 1: ", model_1_rss)
print("\nRSS of Model 2: ", model_2_rss)
print("\nRSS of Model 3: ", model_3_rss)

RSS of Model 1:  971328233540524.8

RSS of Model 2:  961592067853177.8

RSS of Model 3:  905276314555191.6


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [42]:
# Compute the RSS on TESTING data for each of the three models and record the values:
model_1_rss_test = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
model_2_rss_test = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
model_3_rss_test = get_residual_sum_of_squares(model_3, test_data, test_data['price'])

print("RSS of Model 1: ", model_1_rss_test)
print("\nRSS of Model 2: ", model_2_rss_test)
print("\nRSS of Model 3: ", model_3_rss_test)

RSS of Model 1:  226568089091637.8

RSS of Model 2:  224368799992596.34

RSS of Model 3:  251829318996361.94


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.